# !MANUAL!: Stats NZ Infoshare
No automatic way to download the file.

### Steps:
1. http://archive.stats.govt.nz/infoshare/SearchPage.aspx
2. Search for \<Table Code>
3. Open and select all or as described
5. Right hand corner choose .csv
6. Put into the ./drop/ folder.

### Tables: 
1. CPI009AA: CPI All Groups for New Zealand (Qrtly-Mar/Jun/Sep/Dec)
2. CPI017AA: CPI Non-standard All Groups Less/Plus Selected Groupings for New Zealand (Qrtly-Mar/Jun/Sep/Dec)
3. CPI028AA: CPI Level 2 Subgroups for New Zealand, Seasonally adjusted (Qrtly-Mar/Jun/Sep/Dec)
3. CPI029AA: CPI Level 3 Classes for New Zealand, Seasonally adjusted (Qrtly-Mar/Jun/Sep/Dec)
3. DPE054AA: Estimated Resident Population by Age and Sex (1991+) (Qrtly-Mar/Jun/Sep/Dec)
  1. Estimate Type: As at
  2. Population group: Male and Female
  3. Observations: 0 Years, 1 Years, ..., 89 Years, 90 Years and Over
  4. Time: Select All
4. DDE002AA: Estimated Households in Private Occupied Dwellings, As At Quarter Ended (Qrtly-Mar/Jun/Sep/Dec)
5. DDE005AA: Estimated Private Dwellings, As At Quarter Ended (Qrtly-Mar/Jun/Sep/Dec).
6. SNE205AA: Households, Income & Outlay account, Current Prices (Annual-Mar)
7. SNE095AA: Group: National Accounts - SNA 2008 - SNE
8. LAS003AA: Local Authority Statistics - LAS: Income - Seasonally Adjusted (Qrtly-Mar/Jun/Sep/Dec)

In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import requests
import os
import glob   
from shutil import copyfile

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
    
def prepend_history(history, now, kfunc=np.mean):  
    """
    Prepend the history rows to the current rows.
    There should be an intersection.
    It assumes that the current values are more precise. 
    It calculates the correction coefficient on the intersection
    which will be used as a multiplier for all historical value.
    """
    def f(ns, hs):
        ks = ns.combine(hs, lambda n, h: n/h)
        if kfunc == 'oldest':
            k = ks.dropna().head(1).mean()
        else:
            k = kfunc(ks)
        hs = hs.loc[:ns.first_valid_index()]
        if np.isnan(k):
            return ns.combine(hs, lambda n, h: n if not np.isnan(n) else h)
        else:
            return ns.combine(hs, lambda n, h: n if not np.isnan(n) else h*k)
    return now.combine(history, f)  

def read_stats_nz_csv(file, year='march', header=1, na_values=['..', '0.000000'], skiprows_after_header=0, usecols=None, names=None):
    df = pd.read_csv(file)
    rows = df[df.iloc[:, 0] == 'Table information:'].index[0]
    df = pd.read_csv(file, index_col=0, header=header, 
                     skiprows=range(header + 1, header + 1 + skiprows_after_header), 
                     nrows=rows - header - skiprows_after_header,  
                     parse_dates=True, na_values=na_values,
                     usecols=usecols, names=names)
    if year == 'march':
        df.index += pd.DateOffset(months=3)
    return df

if not os.path.exists("drop"):
    os.mkdir("drop")
if not os.path.exists("out"):
    os.mkdir("out")
if not os.path.exists("out/download"):
    os.mkdir("out/download")
if not os.path.exists("out/drop"):
    os.mkdir("out/drop")    
if not os.path.exists("out/tmp"):
    os.mkdir("out/tmp")
if not os.path.exists("out/gen"):
    os.mkdir("out/gen") 

In [2]:
def drop(what, src, dst):
    cpis = glob.glob("drop/" + src)
    src_drop_file = "src/drop/" + dst
    drop_file = "out/drop/" + dst
    if len(cpis) == 1:
        if os.path.exists(drop_file):
            os.remove(drop_file)
        if os.path.exists(src_drop_file):
            os.remove(src_drop_file)
        copyfile(cpis[0], src_drop_file)
        os.rename(cpis[0], drop_file)
        print("Dropped ./" + drop_file)
    elif not os.path.exists(drop_file):
        copyfile(src_drop_file, drop_file)
        print("Copied old file from ./src/drop ./" + drop_file)
        
drop("Consumer price index (CPI)", "CPI316601*.csv", "cpi_stats-nz_1914-now.csv")
drop("Non-standard Consumer price index (CPI)", "CPI318001*.csv", "cpi-nstd_stats-nz_1999-now.csv")
drop("CPI Subgroups", "CPI471301*.csv", "cpi-subgroups-nz_2006-now.csv")
drop("CPI Classes", "CPI471401*.csv", "cpi-classes-nz_2006-now.csv")
drop("Population", "DPE*.csv", "population_stats-nz_1991-now.csv")
drop("Occupied Dwellings", "DDE268801*.csv", "dwellings-occupied_stats-nz_1991-now.csv")
drop("All Dwellings", "DDE269101*.csv", "dwellings-all_stats-nz_1991-now.csv")
drop("Disposable Income", "SNE532901*.csv", "income_stats-nz_1987-now.csv")
drop("Disposable Income 1972", "SNE449101*.csv", "income_stats-nz_1972-now.csv")
drop("Rates", "LAS182601*.csv", "rates_stats-nz_1992-now.csv")

In [3]:
def download(file, url): 
    r = requests.get(url)
    if r.status_code == 200:
        with open('out/download/' + file, 'wb') as f:
            f.write(r.content)
            print("Downloaded ./out/download/" + file)
        copyfile('out/download/' + file, 'src/download/' + file)
    else:
        raise Exception('Cannot download ' + file + ': ' + r.status_code)

download('rent-region_mbie_1993-now.csv', 'https://www.mbie.govt.nz/assets/Data-Files/Building-and-construction/Tenancy-and-housing/Rental-bond-data/Region/region-mean-rents.csv')
download('gdp_rbnz_1987-now.xlsx', 'https://www.rbnz.govt.nz/-/media/ReserveBank/Files/Statistics/tables/m5/hm5.xlsx')
download('mortgage_rbnz_1964-now.xlsx', 'https://www.rbnz.govt.nz/-/media/ReserveBank/Files/Statistics/tables/b3/hb3.xlsx')

Downloaded ./out/download/rent-region_mbie_1993-now.csv
Downloaded ./out/download/gdp_rbnz_1987-now.xlsx
Downloaded ./out/download/mortgage_rbnz_1964-now.xlsx


# CPI Consumers price index (Deflator)

!TODO: Purchase of new houses and rent is included in CPI and it needs to be adjusted

In [4]:
#Quartally March
cpi = read_stats_nz_csv('out/drop/cpi_stats-nz_1914-now.csv')
cpi.index.name = 'Date'
cpi.columns = ['CPI']

cpi['CPI - Housing'] = cpi['CPI']

#Quartally March
cpi_nstd = read_stats_nz_csv('out/drop/cpi-nstd_stats-nz_1999-now.csv')
cpi_nstd = cpi_nstd[['All groups less housing and household utilities group']].dropna()
cpi_nstd.columns = ['CPI - Housing']

cpi = prepend_history(history = cpi, now = cpi_nstd, kfunc='oldest')

cpi = cpi / cpi.iloc[-1]
cpi['Deflator'] = cpi['CPI - Housing']
cpi = cpi.resample('MS').interpolate(method='linear').fillna(method='ffill')


deflator = cpi[['Deflator']]
cpiDeflator = cpi[['CPI']]
cpiDeflator.columns = ['CPIDeflator']

cpi.to_csv('out/gen/cpi_1914-now.csv')
print("Generated ./out/gen/cpi_1914-now.csv")

Generated ./out/gen/cpi_1914-now.csv


# Reinz Indexes

https://www.interest.co.nz/charts/real-estate/median-price-reinz

House prices csv files are downloaded for all regions and total in the house-prices folder and merge the csv files in one ./now/house-prices-reinz-1992-now.csv

In [5]:
regions = pd.read_csv('src/const/nz-regions.csv', index_col=0)
housePricesDirName = 'out/download/house-prices'
housePricesSrcDirName = 'src/download/house-prices'
if not os.path.exists(housePricesDirName):
    os.mkdir(housePricesDirName)

def downloadRegion(code):
    base_url = 'https://www.interest.co.nz/charts-csv/chart_data/real%20estate/medianhouseprice-$code.csv'
    url = base_url.replace('$code', code)
    r = requests.get(url)
    if r.status_code == 200:    
        with open(housePricesDirName + '/' + code + '.csv', 'wb') as f:
            f.write(r.content)
            #print("Downloaded region " + code)
        copyfile(housePricesDirName + '/' + code + '.csv', housePricesSrcDirName + '/' + code + '.csv')
    else:
        raise Exception('Cannot download region ' + code + ': ' + r.status_code)

def downloadHousePrices():
    downloadRegion('total')
    for code in regions.index.values:
        downloadRegion(code)

def loadDf(code, name): 
    df = pd.read_csv(housePricesDirName + '/' + code + '.csv', 
                header=None, names=['Date', name], 
                index_col='Date', parse_dates=True)
    df.index = df.index + pd.DateOffset(1)
    return df
    
def combineCsv():
    dfs = [loadDf(index, r['Name']) for index, r in regions.iterrows()]
    dfs.append(loadDf('total', 'New Zealand'))
    housePrices = pd.concat(dfs, axis='columns')
    housePrices.to_csv('out/gen/house-prices_reinz_1992-now.csv')
    print("Created ./out/gen/house-prices_reinz_1992-now.csv")
        

downloadHousePrices()
combineCsv()

Created ./out/gen/house-prices_reinz_1992-now.csv


# Stats NZ: Long term house prices

http://archive.stats.govt.nz/browse_for_stats/economic_indicators/NationalAccounts/long-term-data-series/prices.aspx

G6.1 Property prices and indexes.xls
http://archive.stats.govt.nz/~/media/Statistics/browse-categories/economic-indicators/national-accounts/Long-term%20data%20series/G%20Prices/table-g6-1.xls

Sheet AREMOS
Located ./src/history/table-g6-1.xls
Year ending December

We join it with the REINZ index and save at ./out/gen/house-prices-1962-now.csv


In [6]:
#Monthly End
houses1992 = pd.read_csv('out/gen/house-prices_reinz_1992-now.csv', index_col='Date', parse_dates=True)
houses1992 = houses1992[['New Zealand']]
houses1992.columns = ['Nominal House Prices']

#Annually December
houses1962 = pd.read_excel(
    'src/history/house-prices_stats-nz_table-g6-1_1962-2004.xls', sheet_name='AREMOS', 
    parse_dates=True, index_col=0, skipfooter=1,  
)
df = houses1962['PQHPI'].to_frame()
df.columns = ['Nominal House Prices']
houses1962['Nominal House Prices'] = houses1962['PQHDR']
houses1962 = houses1962[['Nominal House Prices']]
houses1962 = prepend_history(df, houses1962).dropna()
houses1962

houses = prepend_history(houses1962, houses1992).dropna()
houses = houses.resample('MS').interpolate(method='linear')
houses = pd.merge_asof(houses, deflator, left_index=True, right_index=True)
houses['Deflated House Prices'] = houses['Nominal House Prices']/houses.Deflator
houses.to_csv('out/gen/house-prices_1962-now.csv')

print("Created ./out/gen/house-prices_1962-now.csv")

Created ./out/gen/house-prices_1962-now.csv


# Population

Stats NZ since 1936

In [7]:
#Annual December
pop1936 = read_stats_nz_csv('src/history/population_stats-nz_1936-1995.csv', year='december', header=3)

pop1936['Population'] = pop1936.sum(axis = 1)
pop1936 = pop1936[['Population']]

#Quartally March
pop1991 = read_stats_nz_csv('out/drop/population_stats-nz_1991-now.csv', header=3)
pop1991['Population'] = pop1991.sum(axis = 1)
pop1991 = pop1991[['Population']]

pop = pop1991.combine_first(pop1936)
pop.index.name = 'Date'
pop = pop.resample('MS').interpolate(method='linear')
pop.to_csv('out/gen/population_stats-nz_1936-now.csv')
print("Created ./out/gen/population_stats-nz_1936-now.csv")

Created ./out/gen/population_stats-nz_1936-now.csv


# Dwellings 

Stats NZ since 1961

Count of unoccupied dwellings look like estimated in centuses and then interpolated. It does not have much value.

!TODO: Calculate household considering flats

In [8]:
#Annually March
dw1961 = read_stats_nz_csv('src/history/dwellings_stats-nz_1961-1997.csv', skiprows_after_header=1)
dw1961['Dwellings'] = dw1961.sum(axis=1)
del(dw1961['Unoccupied Dwellings'])

#Quartarlly March
dw1991_occ = read_stats_nz_csv('out/drop/dwellings-occupied_stats-nz_1991-now.csv',
                         usecols=[0,1,3], names=['Date', 'Occupied Dwellings', 'Rented Dwellings']
                        )
#Quartarlly March
dw1991_all = read_stats_nz_csv('out/drop/dwellings-all_stats-nz_1991-now.csv', 
                         usecols=[0,1], names=['Date', 'Dwellings']
                        )

dw1991 = dw1991_all.join(dw1991_occ).dropna()
dwellings = prepend_history(dw1961, dw1991)

dwellings = dwellings.resample('MS').interpolate(method='linear')
dwellings = pd.merge_asof(dwellings, pop, left_index=True, right_index=True)
dwellings['People per Dwelling'] = dwellings['Population']/dwellings['Occupied Dwellings']
dwellings['Occupancy Rate'] = dwellings['Occupied Dwellings']/dwellings['Dwellings']
dwellings['Rented Dwellings Rate'] = dwellings['Rented Dwellings']/dwellings['Occupied Dwellings']
dwellings.to_csv('out/gen/dwellings_stats-nz_1961-now.csv')
print("Created ./out/gen/dwellings_stats-nz_1961-now.csv")

Created ./out/gen/dwellings_stats-nz_1961-now.csv


# MBIE: Rental Bond Data
https://www.mbie.govt.nz/building-and-energy/tenancy-and-housing/rental-bond-data/

Mean rent by region from 1993

To get more historical rents we get src/history/cpi-groups_stats-nz_1975-1999.csv and Dwelling Rentals serias.

The result is saved out/gen/rent_1975-now.csv

In [9]:
#Quartally march
rent1975 = read_stats_nz_csv('src/history/cpi-groups_stats-nz_1975-1999.csv', header=2)
rent1975 = rent1975[['Dwelling Rentals**']]
rent1975.columns = ['Nominal Rent']
rent1975 = rent1975.resample('MS').interpolate(method='linear')

#Monthly start
rent1993 = pd.read_csv('out/download/rent-region_mbie_1993-now.csv', parse_dates=True, index_col=0)
rent1993 = rent1993[['National Total']]
rent1993.columns = ['Nominal Rent']

rent = prepend_history(rent1975, rent1993)
rent = pd.merge_asof(rent, deflator, left_index=True, right_index=True)
rent['Deflated Rent'] = rent['Nominal Rent']/rent.Deflator
rent = rent[['Nominal Rent', 'Deflated Rent', 'Deflator']].dropna()
rent.to_csv('out/gen/rent_1975-now.csv')
print("Created ./out/gen/rent_1975-now.csv")

Created ./out/gen/rent_1975-now.csv


# GDP and private cosumption

RBNZ GDP since 1987
Stats NZ Long term data: http://archive.stats.govt.nz/browse_for_stats/economic_indicators/NationalAccounts/long-term-data-series/prices.aspx

Table E1 for GDP

Table E4 for private consumption

Consumption and GDP includes not only rent but also imputed rent by owner occupied houses:
http://datainfoplus.stats.govt.nz/Item/example.org/87a4fec5-24a8-48fd-a261-3fad8495136e

!TODO: It looks like a part of morgages (here pricipal is mentioned) are included in consumption:
http://archive.stats.govt.nz/browse_for_stats/people_and_communities/Households/HouseholdEconomicSurvey_HOTPYeJun07/Commentary.aspx

Detalization can be found in national accounts: https://www.stats.govt.nz/information-releases/national-accounts-income-and-expenditure-year-ended-march-2019

It's available on infoshare.

In [10]:
#Quartally Date
gdp1987 = pd.read_excel('out/download/gdp_rbnz_1987-now.xlsx', 
                        header=None, skiprows=5, 
                        parse_dates=True, index_col=0,
                        usecols="A,F,N", names=["Date", "GDP", "Consumption"]
                       )
gdp1987 = gdp1987.rolling(4).sum().iloc[3::4]
gdp1987.index = gdp1987.index + pd.DateOffset(days=1)

#Quartally March
cons1949 = pd.read_excel(
        'src/history/private-consumption_stats-nz_table-e4_1949-2004.xls', 
         header=None, skiprows=9,
         parse_dates=True, index_col=0, 
         usecols="A,N", names=["Date", "Consumption"]
    ).dropna()
cons1949.index = cons1949.index + pd.DateOffset(months=3)
cons1949


#Quartally March
gdp1860 = pd.read_excel(
        'src/history/gdp_stats-nz_table-e1_1860-2004.xls', 
         header=None, skiprows=9,
         parse_dates=True, index_col=0, 
         usecols="A,Z", names=["Date", "GDP"]
    ).dropna()
gdp1860.index = gdp1860.index + pd.DateOffset(months=3)
gdp1860


gdp = prepend_history(cons1949, gdp1987)
gdp = prepend_history(gdp1860, gdp)

mln = 1000000
gdp = gdp*mln
gdp = pd.merge_asof(gdp, pop, left_index=True, right_index=True)
gdp = pd.merge_asof(gdp, deflator, left_index=True, right_index=True)
gdp = pd.merge_asof(gdp, rent[["Nominal Rent"]], left_index=True, right_index=True)
gdp = pd.merge_asof(gdp, dwellings[["Occupied Dwellings"]], left_index=True, right_index=True)


gdp["Rent"] = gdp["Nominal Rent"]*365/7*gdp["Occupied Dwellings"]
del(gdp["Nominal Rent"])
#gdp["Consumption - Rent"] = gdp["Consumption"] - gdp["Rent"]
#gdp["GDP - Rent"] = gdp["GDP"] - gdp["Rent"]

gdp["Deflated GDP"] = gdp["GDP"]/gdp.Deflator
gdp["Deflated Consumption"] = gdp["Consumption"]/gdp.Deflator
#gdp["Deflated GDP - Rent"] = gdp["GDP - Rent"]/gdp.Deflator
#gdp["Deflated Consumption - Rent"] = gdp["Consumption - Rent"]/gdp.Deflator

#gdp["GDP per capita"] = gdp["GDP"]*mln/gdp.Population
#gdp["Deflated GDP per capita"] = gdp["Deflated GDP"]*mln/gdp.Population
#gdp["Deflated GDP - Rent per capita"] = gdp["Deflated GDP - Rent"]*mln/gdp.Population
#gdp["Consumption per capita"] = gdp["Consumption"]*mln/gdp.Population
#gdp["Deflated Consumption per capita"] = gdp["Deflated Consumption"]*mln/gdp.Population
#gdp["Deflated Consumption - Rent per capita"] = gdp["Deflated Consumption - Rent"]*mln/gdp.Population

gdp.to_csv("out/gen/gdp_1860-now.csv")
print("Created ./out/gen/gdp_1860-now.csv")

Created ./out/gen/gdp_1860-now.csv


# Disposable Income

Before 1987 there is no data for household income. We can get from salaries.

"Gross operating surplus and gross mixed income" is removed from disposable income because it's the rent payments and imputed credit for the rent: 
https://www.abs.gov.au/AUSSTATS/abs@.nsf/66f306f503e529a5ca25697e0017661f/ac6c11a0f11910fbca2569a40006164b!OpenDocument

Gross disposobale income does not consider houses depreciation. So we need it rather than net.

In [11]:
#Annually March
income1987 = read_stats_nz_csv('out/drop/income_stats-nz_1987-now.csv', header=2, skiprows_after_header=2)
income1987 = income1987[['Gross disposable income', 'Gross operating surplus and gross mixed income']]
income1987['Income'] = income1987['Gross disposable income'] - income1987['Gross operating surplus and gross mixed income']
income1987 = income1987[['Income']]

#Annually March
income1972 = read_stats_nz_csv('out/drop/income_stats-nz_1972-now.csv')
income1972 = income1972[['Compensation Of Employees - received', 'Disposable Income - gross', 'Gross operating surplus and gross mixed income']]
income1972['Income'] = income1972['Disposable Income - gross'] - income1972['Gross operating surplus and gross mixed income']
income1972.columns = ['Salaries', 'x', 'y', 'Income']
income1972 = income1972[['Salaries', 'Income']]


income = prepend_history(income1972, income1987)
income = income*1000000
income = income.resample('MS').interpolate(method='linear')
income = pd.merge_asof(income, pop, left_index=True, right_index=True)
income = pd.merge_asof(income, deflator, left_index=True, right_index=True)
income = pd.merge_asof(income, rent[["Nominal Rent", "Deflated Rent"]], left_index=True, right_index=True)
income = pd.merge_asof(income, dwellings[["Occupied Dwellings"]], left_index=True, right_index=True)

income['Deflated Income'] = income['Income']/income.Deflator
income['Deflated Salaries'] = income['Salaries']/income.Deflator
income['Deflated Household Income'] = income['Deflated Income']/income['Occupied Dwellings']
income['Deflated Rent'] = income['Deflated Rent']*365/7
income['Rent Income Ratio'] = income['Deflated Rent']/income['Deflated Household Income'] 

income.to_csv("out/gen/income_stats-nz_1972-now.csv")
print("Created ./out/gen/income_stats-nz_1972-now.csv")

Created ./out/gen/income_stats-nz_1972-now.csv


# Mortgages

RBNZ, Stats NZ

! TODO: Deposite rates


In [12]:
from mortgage import Loan

#Monthly End
mortgage = pd.read_excel('out/download/mortgage_rbnz_1964-now.xlsx', 
                        header=None, skiprows=5, 
                        parse_dates=True, index_col=0,
                        usecols="A,C,F", names=["Date", "Mortgage Rate", "Deposit Rate"]
                       )
mortgage.index = mortgage.index + pd.DateOffset(days=1)

#Annually March
mortgage1913 = pd.read_excel(
        'src/history/mortgage_stats-nz_table-f3-1_1913-2004.xls', 
         header=None, skiprows=9,
         parse_dates=True, index_col=0, 
         usecols="A,I", names=["Date", "Mortgage Rate"]
    ).dropna()
mortgage1913.index = mortgage1913.index + pd.DateOffset(months=3)
mortgage1913

mortgage = mortgage.combine_first(mortgage1913)
mortgage = mortgage.resample('MS').interpolate(method='linear')
mortgage = mortgage/100
inflation = deflator.pct_change(12)
inflation.columns = ['Inflation']
mortgage = pd.merge_asof(mortgage, inflation, left_index=True, right_index=True)
mortgage['Real Mortgage Rate'] = mortgage['Mortgage Rate'] - mortgage['Inflation']
mortgage.dropna()
mortgage = pd.merge_asof(mortgage, houses, left_index=True, right_index=True)
mortgage.dropna()

def calcMortgage(row): 
    loan = Loan(principal=row['Deflated House Prices']*0.8, interest=row['Mortgage Rate'], term=30)
    return pd.Series([float(loan.monthly_payment*12), float(loan.total_interest)], index=['Deflated Mortgage', 'Deflated Total Mortgage'])
mortgagePayment = mortgage.dropna().apply(calcMortgage, axis=1)
mortgage = mortgage.join(mortgagePayment)
del(mortgage['Deflator'])
mortgage = pd.merge_asof(mortgage, income, left_index=True, right_index=True)
mortgage['Mortgage Income Ratio'] = mortgage['Deflated Mortgage']/mortgage['Deflated Household Income'] 
mortgage['House Prices Income Ratio'] = mortgage['Deflated House Prices']/mortgage['Deflated Household Income'] 
mortgage.to_csv('out/gen/mortgage_1913-now.csv')
print("Created ./out/gen/mortgage_1913-now.csv")


Created ./out/gen/mortgage_1913-now.csv


# Rates

It's unclear how much rates are collected from resedential properties only. Business rates percentage vary highly from region. In Auckland, it's about 35%, in Rotorua, it's 10%. We will assume that 25% is businness rates. Later it can be improved.

Rates, insurance, maintanance, management
Depreciation?

In [13]:
#Quartally Match
rates = read_stats_nz_csv('out/drop/rates_stats-nz_1992-now.csv')
rates = rates[["All rates"]]
rates.columns = ["Rates"]
rates = rates*0.75 *1000 #Remove business rates and $K => $
rates = rates.rolling(4).sum().dropna()
rates = rates.resample('MS').interpolate(method='linear')
rates = pd.merge_asof(dwellings, rates, left_index=True, right_index=True)
rates["Average Rate"] = rates["Rates"]/rates["Dwellings"]
df = cpi[["CPI"]]
df.columns = ["Average Rate"]
rates = prepend_history(history = df, now=rates, kfunc='oldest')
rates["Rates"] = rates["Average Rate"]*rates["Dwellings"]

rates = rates[["Average Rate", "Dwellings", "Rates"]].dropna()
rates = pd.merge_asof(rates, cpiDeflator, left_index=True, right_index=True)
rates["Deflated Average Rate"] = rates["Average Rate"]/rates.CPIDeflator

rates = pd.merge_asof(rates, income, left_index=True, right_index=True)
rates['Average Rate Income Ratio'] = rates['Deflated Average Rate']/rates['Deflated Household Income'] 

rates.to_csv('out/gen/rates_1992-now.csv')
print("Created ./out/gen/rates_1992-now.csv")

Created ./out/gen/rates_1992-now.csv


# Maintenance

Barfoot & Thomson calculated maintenance for rental properties in 2016: https://www.scoop.co.nz/stories/BU1703/S00953/property-maintenance-costs-surprising.htm

CPI Subgroups up to 2006 then standard CPI because there is no suitable group in the earlier CPI

In [14]:
#For 2016    
maitenance = pd.read_csv('src/history/maintenance_barfoot_2016.csv', index_col=0)
house = maitenance.loc['House', 'annual_cost']
cpiSubgroups = read_stats_nz_csv('out/drop/cpi-subgroups-nz_2006-now.csv', header=2)
pm = cpiSubgroups[['Property maintenance']]
maitenance = pm*house/pm.loc['2017-01-01']
maitenance.columns = ['Maintenance']
maitenance = maitenance.resample('MS').interpolate(method='linear')
df = deflator.copy()
df.columns = ['Maintenance']
maitenance = prepend_history(df, maitenance, kfunc='oldest')
maitenance = pd.merge_asof(maitenance, deflator, left_index=True, right_index=True)
maitenance['Deflated Maintenance'] = maitenance['Maintenance']/maitenance.Deflator

maitenance.to_csv('out/gen/maitenance_2006-now.csv')
print("Created ./out/gen/maitenance_2006-now.csv")

Created ./out/gen/maitenance_2006-now.csv


# Insurance

CPI Classes contain insurance inflation up to 2006. There is no data before.

Sample beginning 2011: https://www.interest.co.nz/insurance-data/house/premium-rates

In [15]:
xl = pd.ExcelFile('src/history/insurance_interest-co-nz_2011.xlsx')
dfs = [xl.parse(sheet_name, header=1, skiprows=range(2, 7)) for sheet_name in xl.sheet_names]
df = pd.concat(dfs)
df = df.melt(id_vars=['Company', 'Plan name', 'Product\nScore', 'Excess\n$'])
insurance2011 = df.value.mean()

cpi_classes = read_stats_nz_csv('out/drop/cpi-classes-nz_2006-now.csv', header=2)
pm = cpi_classes[['Dwelling insurance']]
insurance = pm*insurance2011/pm.loc['2011-01-01']
insurance.columns = ['Insurance']
insurance = insurance.resample('MS').interpolate(method='linear')

df = deflator.copy()
df.columns = ['Insurance']
insurance = prepend_history(df, insurance, kfunc='oldest')

insurance = pd.merge_asof(insurance, deflator, left_index=True, right_index=True)
insurance['Deflated Insurance'] = insurance['Insurance']/maitenance.Deflator
insurance.to_csv('out/gen/insurance_2006-now.csv')
print("Created ./out/gen/insurance_2006-now.csv")


Created ./out/gen/insurance_2006-now.csv


# Model

$rent - deposit*depositInterest' >= (propertyCost - deposit)*morgageInterest + expenses - gain'$

$depositInterest' = depositInterest*(1-tax) - inflation$

$gain' = propertyCost*(properyInflation - inflation)$

$deposit = propertyCost*depositRate$

$expenses = rates + insurance + maintenance$

Parameters:
* rent - av annual rent
* depositRate - deposit percentage, default min 20%
* depositInterest - 6 month deposit interest
* tax - income tax = 0.28
* inflation - annual products inflation excluding rent
* propertyCost - av property cost


In [28]:
model = rates[['Deflated Average Rate']]
model = pd.merge_asof(model, insurance[['Deflated Insurance']], left_index=True, right_index=True)
model = pd.merge_asof(model, maitenance[['Deflated Maintenance']], left_index=True, right_index=True)
model = pd.merge_asof(model, rent[['Deflated Rent']], left_index=True, right_index=True)
model = pd.merge_asof(model, mortgage[['Mortgage Rate', 'Deposit Rate']], left_index=True, right_index=True)
model = pd.merge_asof(model, houses[['Deflated House Prices']], left_index=True, right_index=True)
model = pd.merge_asof(model, cpi[['CPI - Housing']].pct_change(12), left_index=True, right_index=True)
model = model.rename({
    'Deflated Average Rate': 'rates', 
    'Deflated Insurance': 'insurance', 
    'Deflated Maintenance': 'maintenance',
    'Deflated Rent': 'rent',
    'Mortgage Rate': 'mortgage_interest',
    'Deposit Rate': 'deposit_interest',
    'Deflated House Prices': 'property_cost',
    'CPI - Housing': 'inflation'
}, axis=1)

model = model.dropna()
model.rent = model.rent*365/7
model['tax'] = 0.28
model['expenses'] = model.rates + model.insurance + model.maintenance
model['management'] = model.rent*0.08
model['investor_expenses'] = model.management + model.rates + model.insurance*1.5 + model.maintenance
model['expenses_rate'] = model.expenses/model.property_cost
model['investor_expenses_rate'] = model.expenses/model.property_cost
model['rent_interest'] = model.rent/model.property_cost
model['property_inflation'] = model.inflation
model['gain_rate'] = model.property_inflation - model.inflation
model['deposit_interest_adj'] = model.deposit_interest*(1-tax) - model.inflation

model.to_csv('out/gen/model_1975-now.csv')
print("Created ./out/gen/model_1975-now.csv")

Created ./out/gen/model_1975-now.csv


# !TODO: Issued Money (M2)

M2, money velocity